# 1. Setup and Load Data

## 1.1 Install Dependencies and Setup

In [ ]:
%pip install tensorflow opencv-python matplotlib

In [6]:
import tensorflow as tf
import os

2024-12-06 23:29:44.145712: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)